In [3]:
pip install scikit-learn


   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
    --------------------------------------- 0.3/11.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.2 MB 1.0 MB/s eta 0:00:11
   -- ------------------------------------- 0.8/11.2 MB 985.5 kB/s eta 0:00:11
   --- ------------------------------------ 1.0/11.2 MB 1.1 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/11.2 MB 1.1 MB/s eta 0:00:10
   ----- ---------------------------------- 1.6/11.2 MB 1.1 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.2 MB 1.1 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/11.2 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.2 MB 1.1 MB/s eta 0:00:08
   --------- ------------------------------ 2.6/11.2 MB 1.2 MB/s eta 0:00:08
   ---------- ----------------------------- 2.9/11.2 MB 1.2 MB/s eta 0:00:07
   ------------ -

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt

# Define the U-Net model
def unet_model(input_shape=(128, 128, 12), num_classes=5):
    """
    U-Net model for land terrain classification.

    Args:
        input_shape: Shape of the input image (height, width, channels).
        num_classes: Number of terrain classes for classification.

    Returns:
        Compiled U-Net model.
    """
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder: Contracting path
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder: Expanding path
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs, outputs)
    return model


# Data preparation
# def load_data(image_folder, image_size=(128, 128)):
#     """
#     Load and preprocess images from a folder.

#     Args:
#         image_folder: Path to the folder containing images.
#         image_size: Target size for resizing images.

#     Returns:
#         Numpy array of images.
#     """
#     images = []
#     labels = []  # Placeholder if you have labels
#     for image_name in os.listdir(image_folder):
#         img_path = os.path.join(image_folder, image_name)
#         img = imread(img_path)
#         img = resize(img, image_size, mode='constant', preserve_range=True)
#         images.append(img)
#     return np.array(images)

def load_data(image_folder, image_size=(128, 128), num_channels=12):
    images = []
    for image_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, image_name)
        img = imread(img_path)
        img = resize(img, image_size + (num_channels,), mode='constant', preserve_range=True)
        images.append(img)
    return np.array(images)



# Main setup
input_shape = (128, 128, 12)  # Adjust according to your Sentinel-2 data
num_classes = 5

train_folder = 'C://Users/maham/OneDrive/Desktop/forest_train/'  # Update with your actual path
test_folder = 'C://Users/maham/OneDrive/Desktop/forest_test/'  # Update with your actual path

# Load and preprocess data
X_train = load_data(train_folder, image_size=input_shape[:2])
X_test = load_data(test_folder, image_size=input_shape[:2])

# Normalize data
X_train = X_train / 255.0
X_test = X_test / 255.0

# Generate dummy labels (Replace with actual labels if available)
Y_train = np.random.randint(0, num_classes, (len(X_train), input_shape[0], input_shape[1]))
Y_test = np.random.randint(0, num_classes, (len(X_test), input_shape[0], input_shape[1]))

# One-hot encoding labels
Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=num_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, num_classes=num_classes)

# Split training data into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Build and compile the model
model = unet_model(input_shape=input_shape, num_classes=num_classes)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model and collect accuracy/loss data
history = model.fit(
    X_train,
    Y_train,
    validation_data=(X_val, Y_val),
    batch_size=16,
    epochs=50,
    verbose=1
)

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=1)

# Plot training and validation accuracy and loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss')

plt.show()

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Epoch 1/50
100/100 [==============================] - 1156s 12s/step - loss: 1.6095 - accuracy: 0.2001 - val_loss: 1.6094 - val_accuracy: 0.2001
Epoch 2/50
 52/100 [==============>...............] - ETA: 7:27 - loss: 1.6094 - accuracy: 0.1999


KeyboardInterrupt

